In [ ]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_train, input_list_train, input_list_test, output_list_test
from eval import extracting_steering_vector, calc_loss_steering_vector

ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"

In [ ]:
generator = Llama.build(ckpt_dir=ckpt_dir, tokenizer_path=tokenizer_path, max_seq_len= 1024, max_batch_size= 4, activation=True, activation_layer=12)
print(generator)

In [ ]:
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= generator.model.params.max_seq_len - 1

In [ ]:
#extracting a steering vector for every layer
for k in range(generator.model.layers):
    steering_vector = extracting_steering_vector(generator, layer=k, iter=2000)
    torch.save(steering_vector, f"./steering_vectors/vector{k}.pt")

In [ ]:
#calculating the loss for each layes
losses = {}
for k in range(generator.model.layers):
    steering_vec = torch.load(f"./steering_vectors/vector{k}.pt")
    loss = calc_loss_steering_vector(generator, layer=k, iter=len(input_list_test))
    losses[k] = loss
